# Classification based on Quickdraw Dataset

### Imports

In [71]:
import random
import json
import numpy as np
import cairocffi as cairo
import os
from PIL import Image

### The Dataset

In [72]:
# Get the classes that will be used
def load_classes(file_path):
    res = {}
    count = 0
    for line in open(file_path, 'r'):
        res[line] = count
        count+=1
    return res

In [73]:
### The code is taken from the original GitHub of the QuickDrawDataset
def vector_to_raster(vector_images, side=64, line_diameter=16, padding=16, bg_color=(0,0,0), fg_color=(1,1,1)):
    """
    padding and line_diameter are relative to the original 256x256 image.
    """
    
    original_side = 256.
    
    surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, side, side)
    ctx = cairo.Context(surface)
    ctx.set_antialias(cairo.ANTIALIAS_BEST)
    ctx.set_line_cap(cairo.LINE_CAP_ROUND)
    ctx.set_line_join(cairo.LINE_JOIN_ROUND)
    ctx.set_line_width(line_diameter)

    # scale to match the new size
    # add padding at the edges for the line_diameter
    # and add additional padding to account for antialiasing
    total_padding = padding * 2. + line_diameter
    new_scale = float(side) / float(original_side + total_padding)
    ctx.scale(new_scale, new_scale)
    ctx.translate(total_padding / 2., total_padding / 2.)

    raster_images = []
    for vector_image in vector_images:
        # clear background
        ctx.set_source_rgb(*bg_color)
        ctx.paint()
        
        bbox = np.hstack(vector_image).max(axis=1)
        offset = ((original_side, original_side) - bbox) / 2.
        offset = offset.reshape(-1,1)
        centered = [stroke + offset for stroke in vector_image]

        # draw strokes, this is the most cpu-intensive part
        ctx.set_source_rgb(*fg_color)        
        for xv, yv in centered:
            ctx.move_to(xv[0], yv[0])
            for x, y in zip(xv, yv):
                ctx.line_to(x, y)
            ctx.stroke()

        data = surface.get_data()
        raster_image = np.copy(np.asarray(data)[::4])
        raster_images.append(raster_image)
    
    return raster_images

In [74]:
def load_data(sample, n_images, dimension):
    sample_data = [json.loads(line) for line in open(f'data/full_simplified_{sample}.ndjson', 'r')]
    sample_data = random.choices(sample_data, k=n_images)
    vector_images = [drawing_data['drawing'] for drawing_data in sample_data]
    return vector_to_raster(vector_images, side=dimension)

In [83]:
def save_png(drawing_class, data, dimension):
    if not os.path.exists(f"images/{drawing_class}"): 
        os.makedirs(f"images/{drawing_class}")
    
    count = 0
    for image_arr in data:
        image_arr = np.reshape(image_arr, (dimension, -1))
        img = Image.fromarray(image_arr, "L")
        img.save(f"images/{drawing_class}/axe_{count}.png")
        count += 1

In [85]:
img_dim = 64
classes = load_classes("class_names.txt")
for drawing_class in classes:
    data = load_data(drawing_class, 2, dimension = img_dim)
    save_png(drawing_class, data, dimension = img_dim)
    

For the datas :
- load in arrays
- sort to keep only the recognized drawings
- (save the arrays)
- (possibility to save in png)